In [1]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations
from os import environ

In [2]:
environ['ROS_MASTER_URI'] = 'http://021607CP00116.local:11311'

In [3]:
environ['ROS_HOSTNAME'] = 'mingew.local'

In [4]:
rospy.init_node("sawyer_example")

In [5]:
effector = "right_hand"
world = "world"

In [6]:
tfl = tf.TransformListener(interpolate=True, cache_time=rospy.Duration(1000))
tfb = tf.TransformBroadcaster()

In [7]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [8]:
rs.enable()

[INFO] [1555255256.171759]: Robot Enabled


In [9]:
limb = intera_interface.Limb('right')

# Compute sticks in cartesian space

In [10]:
# Base is ["lower"]["left"][0] for HOPE
base_position = [-0.5, -0.7, 0.35]


In [11]:
# Starting from lower left (drawing direction)
# 
sticks = {"hope": [], "fear": []}
y_interval = 0.01     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.01  # Interval between each group in y direction
num_groups = 8         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.03    # Length in z direction
num_height = 8         # Number of sticks in z direction (in height)
num_points_cart = 1   # Number of cartesian points for the actual drawing motion

emotion = "fear"
z_stick = base_position[2]
col = 0
for h in range(num_height):
    y_stick = base_position[1]
    for g in range(num_groups):
        for i in range(num_sticks):
            # Add the end stick
            end_stick = [base_position[0], y_stick, z_stick + height_stick]
            tfb.sendTransform(end_stick, [0, 0, 0, 1], rospy.Time.now(), str(col), "world")
            motion = [end_stick]

            # Add the cartesian motion
            for p in range(num_points_cart):
                motion_point = deepcopy(end_stick)
                motion_point[2] = z_stick + height_stick - height_stick * (p+1) / num_points_cart
                motion.append(motion_point)
                #if i == 0 and g == 0:
                #    tfb.sendTransform(motion_point, [0, 0, 0, 1], rospy.Time.now(), str(col)+'-'+str(p), "world")
            
            sticks[emotion].append(motion)
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
    if len(sticks["fear"]) == num_groups*num_height*num_sticks/2:
        emotion = "hope"
        z_stick += 0.02
print(len(sticks["hope"]), "+", len(sticks["fear"]), "sticks generated")

(128, '+', 128, 'sticks generated')


In [12]:
sticks['fear'][0]

[[-0.5, -0.7, 0.38], [-0.5, -0.7, 0.35]]

# Cartesian sticks

In [13]:
hand_T_stick = [[0.14239612794977008, 0.01701314337065485, -0.07630844138995452],
 [0.5270677868272005,
  0.023880201241985057,
  -0.8461224120032248,
  0.07553904939543005]]

In [14]:
stick_fk = {"hope" : [], "fear": []}
for emotion in stick_fk:
    for stick_id, stick in enumerate(sticks[emotion]):
        stamp = rospy.Time.now()
        points = []
        for point_id, point in enumerate(stick):
            stamp += rospy.Duration(point_id*5)
            hand_T_world = transformations.multiply_transform([point, [0,0,0,1]], hand_T_stick)
            tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "eik", world)
            points.append(transformations.list_to_pose_stamped(hand_T_world, frame_id="world"))
        stick_fk[emotion].append(points)
        #rospy.sleep(0.1)

In [15]:
stick_fk["hope"][0]

[header: 
   seq: 0
   stamp: 
     secs: 0
     nsecs:         0
   frame_id: "world"
 pose: 
   position: 
     x: -0.3576038720502299
     y: -0.6829868566293451
     z: 0.5236915586100455
   orientation: 
     x: 0.5270677868272005
     y: 0.023880201241985057
     z: -0.8461224120032248
     w: 0.07553904939543005, header: 
   seq: 0
   stamp: 
     secs: 0
     nsecs:         0
   frame_id: "world"
 pose: 
   position: 
     x: -0.3576038720502299
     y: -0.6829868566293451
     z: 0.49369155861004543
   orientation: 
     x: 0.5270677868272005
     y: 0.023880201241985057
     z: -0.8461224120032248
     w: 0.07553904939543005]

In [16]:
tfb.sendTransform(transformations.pose_to_list(stick_fk['hope'][0][0])[0],
                  transformations.pose_to_list(stick_fk['hope'][0][0])[1], rospy.Time.now(), "eik", world)

In [17]:
from intera_motion_interface import (
    MotionTrajectory,
    MotionWaypoint,
    MotionWaypointOptions
)

In [18]:
from intera_motion_msgs.msg import TrajectoryOptions

In [27]:
traj  = MotionTrajectory(limb=limb)

In [28]:
joint_angles = limb.joint_ordered_angles()

In [45]:
waypoint = MotionWaypoint(limb=limb)
waypoint.set_joint_angles(joint_angles)
traj.append_waypoint(waypoint)

In [21]:
waypoint = MotionWaypoint(limb=limb)
waypoint.set_cartesian_pose(stick_fk["hope"][0][0], joint_angles=joint_angles)
traj.append_waypoint(waypoint)

In [22]:
waypoint = MotionWaypoint(limb=limb)
waypoint.set_cartesian_pose(stick_fk["hope"][0][1], joint_angles=joint_angles)
traj.append_waypoint(waypoint)

In [23]:
opts = TrajectoryOptions(interpolation_type=TrajectoryOptions.CARTESIAN)

In [24]:
traj.set_trajectory_options(opts)

In [46]:
traj.send_trajectory(timeout=3)

result: True
errorId: ''
trajectory_analysis: 
  planned_duration: 0.007
  measured_duration: 0.009379551
  min_angle_command: [1.6275419921875, -2.6035751953125, -3.030283203125, 0.9002783203125, 1.14769921875, -1.8767021484375, 0.6463701171875]
  max_angle_command: [1.6275419921875, 1.1754943508222875e-38, 1.1754943508222875e-38, 0.9002783203125, 1.14769921875, 1.1754943508222875e-38, 0.6463701171875]
  peak_speed_command: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  peak_accel_command: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  peak_jerk_command: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  min_time_rate: 1.0
  max_time_rate: 1.0
  max_position_error: [0.0007197265625000782, 0.00010253906250001776, 0.0, 0.0, 0.0, 0.00020605468750001243, 0.0]
  max_velocity_error: [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
last_successful_waypoint: 13

In [30]:
traj.to_msg()

label: "default"
joint_names: [right_j0, right_j1, right_j2, right_j3, right_j4, right_j5, right_j6]
waypoints: 
  - 
    joint_positions: [1.6275419921875, -2.6035751953125, -3.030283203125, 0.9002783203125, 1.14769921875, -1.8767021484375, 0.6463701171875]
    active_endpoint: "right_hand"
    pose: 
      header: 
        seq: 0
        stamp: 
          secs: 0
          nsecs:         0
        frame_id: ''
      pose: 
        position: 
          x: 0.0
          y: 0.0
          z: 0.0
        orientation: 
          x: 0.0
          y: 0.0
          z: 0.0
          w: 0.0
    options: 
      label: "default"
      max_joint_speed_ratio: 0.7
      joint_tolerances: []
      max_joint_accel: [7.0, 5.0, 8.0, 8.0, 8.0, 8.0, 8.0]
      max_linear_speed: 0.6
      max_linear_accel: 0.6
      max_rotational_speed: 1.57
      max_rotational_accel: 1.57
      corner_distance: 0.0
trajectory_options: 
  interpolation_type: "JOINT"
  interaction_control: False
  interaction_params: 
   

# Overwrite other poses

In [ ]:
# Pause pose
poses = {"pause": limb.joint_angles()}

with open("../config/poses.json", "w") as f:
    json.dump(poses, f, indent=4)

In [17]:
with open("../config/poses.json") as f:
    poses = json.load(f)

In [ ]:
limb.move_to_joint_positions(poses["pause"])

In [18]:
poses

{u'pause': {u'right_j0': 2.462169921875,
  u'right_j1': -3.327517578125,
  u'right_j2': -2.2095576171875,
  u'right_j3': -1.52369140625,
  u'right_j4': 0.1305107421875,
  u'right_j5': -1.00734375,
  u'right_j6': -0.2455576171875}}

# Display sticks

In [ ]:
with open("../config/sticks_target_ik.json") as f:
    sticks = json.load(f)

In [ ]:
from sensor_msgs.msg import JointState
p = rospy.Publisher("display/joint_states", JointState, queue_size=1)

In [ ]:
type = 'fear'
side = 'end'
for i, pose in enumerate(sticks[type]):
    js = JointState()
    js.header.stamp = rospy.Time.now()
    js.name = pose[side].keys()
    js.position = pose[side].values()
    p.publish(js)
    input = raw_input('Press to display {} pose {} of [q to quit] '.format(side, i, type))
    if input == 'q': break

In [ ]:
sticks